# Lab 8-2: Dyna-Q and Prioritized Sweeping

Welcome to this Lab! In this notebook, you will:
1. implement the Dyna-Q with and without Prioritized Sweeping!
2. compare their performance on different scenarios

We will give you the environment and infrastructure to run the experiment and visualize the performance. These libraries need to be imported for the repository.

The assignment will be graded automatically by comparing the behavior of your agent to our implementations of the algorithms. The random seed will be set explicitly to avoid different behaviors due to randomness. 

Please go through the cells in order. 

# Clone to Repository
All of the required files for this lab are placed in a repository. So, you need to clone the notebook to this repository in order to add th provided libraries and files to the Contents of this notebook.

In [ ]:
# Clone the notebook to the other required libraries for this lab
!git clone https://github.com/mdehghani86/MazeExampleRep.git

## The Shortcut Maze Environment

In this maze environment, the goal is to reach the goal state (G) as fast as possible from the starting state (S). There are four actions – up, down, right, left – which take the agent deterministically from a state to the corresponding neighboring states, except when movement is blocked by a wall (denoted by grey) or the edge of the maze, in which case the agent remains where it is. The reward is +1 on reaching the goal state, 0 otherwise. On reaching the goal state G, the agent returns to the start state S to being a new episode. This is a discounted, episodic task with $\gamma = 0.95$.

<img src="MazeExampleRep/images/shortcut_env.png" alt="environment" width="400"/>

Later in the assignment, we will use a variant of this maze in which a 'shortcut' opens up after a certain number of timesteps. We will test if the the Dyna-Q and Dyna-Q+ agents are able to find the newly-opened shorter route to the goal state.

## Packages

We import the following libraries that are required for this assignment. Primarily, we shall be using the following libraries:
1. numpy: the fundamental package for scientific computing with Python.
2. matplotlib: the library for plotting graphs in Python.
3. RL-Glue: the library for reinforcement learning experiments.
4. queue: to maitaine "prioritized queue". For its documentaiton visit [here](https://python.readthedocs.io/en/latest/library/asyncio-queue.html#)

**Please do not import other libraries** — this will break the autograder.

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
!pip install jdc
import os, jdc, shutil
from tqdm import tqdm
from queue import PriorityQueue

!pip install rlglue
from MazeExampleRep.rl_glue import RLGlue
from MazeExampleRep.agent import BaseAgent
from MazeExampleRep.maze_env import ShortcutMazeEnvironment

os.makedirs('results', exist_ok=True)

In [ ]:
plt.rcParams.update({'font.size': 15})
plt.rcParams.update({'figure.figsize': [8,5]})

## Section 1: Dyna-Q with "Prioritized Sweeping"

Let's start with a quick recap of the Prioritized sweeping Pseudocode.

<div style="width:80%"><img src="MazeExampleRep/images/prioritized_sweeping.png" alt="DynaQ_pseudocode"></div>

Priority Sweeping involves five basic steps:
1. State/Action Selection: select a nonterminal state and an action to be performed.
2. Model learning: using the observed next state and reward, update the model (here, updating a table as the environment is assumed to be deterministic).
3. Prioritizing: A queue is maintained of every state–action pair whose estimated value would change nontrivially if updated , prioritized by the size of the change. If the effects of changes are efficiently propagated backward until quiescence
4. Planning/Sweeing: update the action values by generating $n$ simulated experiences selected form top of the queue and perform one-step tabular Q-planning method. 
5. Update Queue: the effects of changes are efficiently propagated backward until quiescence

Please check [8.4 Prioritized Sweeping](http://www.incompleteideas.net/book/RLbook2018.pdf#page=191) for more detials.

Let's break this down in pieces and do it one-by-one.

PriorityAgent has the same structure as the DynaQAgent with three major differences. 
In the init function, we initialised a threshold $\theta$ , a priority queue to store state and action pairs by their priority, and a dictionary of predecessors in order to update all states lead to the current updated state.


In [ ]:
# Do not modify this cell!
#PriorityAgent with "Prioritized Sweeping"

class PriorityAgent(BaseAgent):

    def agent_init(self, agent_info):
        """Setup for the agent called when the experiment first starts.

        Args:
            agent_init_info (dict), the parameters used to initialize the agent. The dictionary contains:
            {
                num_states (int): The number of states,
                num_actions (int): The number of actions,
                epsilon (float): The parameter for epsilon-greedy exploration,
                step_size (float): The step-size,
                discount (float): The discount factor,
                planning_steps (int): The number of planning steps per environmental interaction
                step_size (delta): small threshold used for the Prioritized queue,
                random_seed (int): the seed for the RNG used in epsilon-greedy
                planning_random_seed (int): the seed for the RNG used in the planner
            }
        """

        # First, we get the relevant information from agent_info 
        # NOTE: we use np.random.RandomState(seed) to set the two different RNGs
        # for the planner and the rest of the code
        try:
            self.num_states = agent_info["num_states"]
            self.num_actions = agent_info["num_actions"]
        except:
            print("You need to pass both 'num_states' and 'num_actions' \
                   in agent_info to initialize the action-value table")
        self.gamma = agent_info.get("discount", 0.95)
        self.step_size = agent_info.get("step_size", 0.1)
        self.epsilon = agent_info.get("epsilon", 0.1)
        self.planning_steps = agent_info.get("planning_steps", 10)

        self.rand_generator = np.random.RandomState(agent_info.get('random_seed', 50))
        self.planning_rand_generator = np.random.RandomState(agent_info.get('planning_random_seed', 50))

        # Next, we initialize the attributes required by the agent, e.g., q_values, model, etc.
        # A simple way to implement the model is to have a dictionary of dictionaries, 
        #        mapping each state to a dictionary which maps actions to (reward, next state) tuples.
        self.q_values = np.zeros((self.num_states, self.num_actions))
        self.actions = list(range(self.num_actions))
        self.past_action = -1
        self.past_state = -1
        self.model = {} # model is a dictionary of dictionaries, which maps states to actions to 
                        # (reward, next_state) tuples
            
            
        # for priority sweeping
        self.theta = agent_info.get("theta", 0.05)
        self.queue = PriorityQueue()
        self.predecessors = {}  # s' -> list[(s, a)...]

Updating model is exactly the same for both PriorityAgent, and DynaQAgent.
It takes a (s, a, s', r) tuple and stores the next state and reward corresponding to a state-action pair.

In [ ]:
%%add_to PriorityAgent

# [GRADED]

def update_model(self, past_state, past_action, state, reward):
    """updates the model 
    
    Args:
        past_state       (int): s
        past_action      (int): a
        state            (int): s'
        reward           (int): r
    Returns:
        Nothing
    """
    # Update the model with the (s,a,s',r) tuple (1~4 lines)
    
    ### START CODE HERE ###
    
    ### END CODE HERE ###

In [ ]:
# Do not modify this cell!

## Test code for update_model() ##

actions = []
agent_info = {"num_actions": 4, 
              "num_states": 3, 
              "epsilon": 0.1, 
              "step_size": 0.1, 
              "discount": 1.0, 
              "random_seed": 0,
              "planning_random_seed": 0}
test_agent = PriorityAgent()
test_agent.agent_init(agent_info)
test_agent.update_model(0,2,0,1)
test_agent.update_model(2,0,1,1)
test_agent.update_model(0,3,1,2)
print("Model: \n", test_agent.model)

In [ ]:
%%add_to PriorityAgent

# [GRADED]
# update predecessors 1~4 lines

def update_predecessors(self, past_state, past_action, state):
    """updates the predecessors
    
    Args:
        past_state       (int): s
        past_action      (int): a
        state            (int): s'
    Returns:
        Nothing
    """
    ### START CODE HERE ###
    
    ### END CODE HERE ###

In [ ]:
# Do not modify this cell!

## Test code for update_predecessors() ##
actions = []
agent_info = {"num_actions": 4, 
              "num_states": 4, 
              "epsilon": 0.1, 
              "step_size": 0.1, 
              "discount": 0.9, 
              "planning_steps": 5,
              "random_seed": 0,
              "planning_random_seed": 5,
              "theta" : 0.05}

test_agent = PriorityAgent()
test_agent.agent_init(agent_info)

test_agent.update_predecessors(2,1,0)
test_agent.update_predecessors(2,2,2)
test_agent.update_predecessors(2,3,2)
test_agent.update_predecessors(1,1,2)
test_agent.update_predecessors(1,4,3)

# Print the queue items one by one (based on priority)
print(test_agent.predecessors)

In [ ]:
%%add_to PriorityAgent

# [GRADED]

def update_queue(self, past_state, past_action, state, reward):
    """updates the model 
    
    Args:
        past_state       (int): s
        past_action      (int): a
        state            (int): s'
        reward           (int): r
    Returns:
        Nothing
    """
       
    # Step 1: Find the absolute value of the difference between the current value of s in self.values and the highest
        # Q-value across all possible actions from s (this represents what the value should be); 
        # call this number DynaQ_psDelta. Do NOT update self.values[s] in this step.
    # Step 2: Update self.Pqueue : Push s and a into the priority queue with priority -diff (note that this is negative).
        # We use a negative because the priority queue is a min heap, but we want to prioritize updating states that have
        # a higher error.
    
    ### START CODE HERE ### 2-4 lines
    
    ### END CODE HERE ###

In [ ]:
# Do not modify this cell!

## Test code for update_queue() ##
actions = []
agent_info = {"num_actions": 4, 
              "num_states": 4, 
              "epsilon": 0.1, 
              "step_size": 0.1, 
              "discount": 0.9, 
              "planning_steps": 5,
              "random_seed": 0,
              "planning_random_seed": 5,
              "theta" : 0.05}
test_agent = PriorityAgent()

test_agent.agent_init(agent_info)
test_agent.update_queue(0,2,1,1)
test_agent.update_queue(0,1,3,0)
test_agent.update_queue(2,2,3,2)
test_agent.update_queue(3,1,2,3)

# Print the queue items one by one (based on priority)
# Do not this part, loop for is not needed
print(test_agent.queue.get()[1])
print(test_agent.queue.get()[1])
print(test_agent.queue.get()[1])

In [ ]:
%%add_to PriorityAgent

# [GRADED]

def planning_step(self):
    """performs planning, i.e. indirect RL.

    Args:
        None
    Returns:
        Nothing
    """
    
    # The indirect RL step:
    # - Choose a state and action from the queue (~2 lines)
    # - Query the model with this state-action pair for the predicted next state and reward.(~1 line)
    # - Update the action values with this simulated experience.                            (2~4 lines)
    # - Repeat until the queue is empty.
    #
    # Note that the update equation is different for terminal and non-terminal transitions. 
    # To differentiate between a terminal and a non-terminal next state, assume that the model stores
    # the terminal state as a dummy state like -1
    #
    # Important: remember you have a random number generator 'planning_rand_generator' as 
    #     a part of the class which you need to use as self.planning_rand_generator.choice()
    #     For the sake of reproducibility and grading, *do not* use anything else like 
    #     np.random.choice() for performing search control.

    ### START CODE HERE ###    
    # Planning loop
   
        #Part 1: state-action selection from top of the queue
      
        #Part 2: Query the model (s,a)-> (s',r)
       
        #Part 3: Plannig update
           
        
        #Part 4: loop for all state, action predicted lead to _state
       

    ### END CODE HERE ###

In [ ]:
# Do not modify this cell!

## Test code for planning_step() ##
actions = []
agent_info = {"num_actions": 4, 
              "num_states": 6, 
              "epsilon": 0.1, 
              "step_size": 0.1, 
              "discount": 0.9, 
              "planning_steps": 5,
              "random_seed": 0,
              "planning_random_seed": 5,
              "theta" : 0.05}
test_agent = PriorityAgent()
test_agent.agent_init(agent_info)

test_agent.update_model(0,2,1,1)
test_agent.update_queue(0,2,1,1)
test_agent.update_predecessors(0,2,1)

test_agent.update_model(0,1,3,0)
test_agent.update_queue(0,1,3,0)
test_agent.update_predecessors(0,1,3)

test_agent.update_model(2,2,5,2)
test_agent.update_queue(2,2,5,2)
test_agent.update_predecessors(2,2,5)

test_agent.update_model(3,1,2,1)
test_agent.update_queue(3,1,2,1)
test_agent.update_predecessors(3,1,2)

test_agent.planning_step()
print("q_values", test_agent.q_values)

In [ ]:
%%add_to PriorityAgent

# Do not modify this cell!

def argmax(self, q_values):
    """argmax with random tie-breaking
    Args:
        q_values (Numpy array): the array of action values
    Returns:
        action (int): an action with the highest value
    """
    top = float("-inf")
    ties = []

    for i in range(len(q_values)):
        if q_values[i] > top:
            top = q_values[i]
            ties = []

        if q_values[i] == top:
            ties.append(i)

    return self.rand_generator.choice(ties)

def choose_action_egreedy(self, state):
    """returns an action using an epsilon-greedy policy w.r.t. the current action-value function.

    Important: assume you have a random number generator 'rand_generator' as a part of the class
                which you can use as self.rand_generator.choice() or self.rand_generator.rand()

    Args:
        state (List): coordinates of the agent (two elements)
    Returns:
        The action taken w.r.t. the aforementioned epsilon-greedy policy
    """

    if self.rand_generator.rand() < self.epsilon:
        action = self.rand_generator.choice(self.actions)
    else:
        values = self.q_values[state]
        action = self.argmax(values)

    return action

In [ ]:
%%add_to PriorityAgent

# [GRADED]

def agent_start(self, state):
    """The first method called when the experiment starts, 
    called after the environment starts.
    Args:
        state (Numpy array): the state from the
            environment's env_start function.
    Returns:
        (int) the first action the agent takes.
    """
    
    # given the state, select the action using self.choose_action_egreedy()), 
    # and save current state and action (~2 lines)
    ### self.past_state = ?
    ### self.past_action = ?

    ### START CODE HERE ###
   
    ### END CODE HERE ###
    
    return self.past_action

def agent_step(self, reward, state):
    """A step taken by the agent.

    Args:
        reward (float): the reward received for taking the last action taken
        state (Numpy array): the state from the
            environment's step based on where the agent ended up after the
            last step
    Returns:
        (int) The action the agent takes given this state.
    """
    
    # - Direct-RL step (~1-3 lines)
    # - Model Update step (~1 line)
    # - planning_step` (~1 line)
    # - Action Selection step (~1 line)
    # Save the current state and action before returning the action to be performed. (~2 lines)

    ### START CODE HERE ###
    #Part 1: last state - last action
   
    #Part 2:  Update Model  
    
    # Part 3: Update Queue

    # Part 4: Update predecessors
    
    #Part 5: Planning_step

    #Part 6: Action Selection step (~1 line)

    #Part 5: Save the current state and action

    ### END CODE HERE ###
    return self.past_action

def agent_end(self, reward):
    """Called when the agent terminates.

    Args:
        reward (float): the reward the agent received for entering the
            terminal state.
    """
    
    # - Direct RL update with this final transition (1~2 lines)
    # - Model Update step with this final transition (~1 line)
    # - One final `planning_step` (~1 line)
    #
    # Note: the final transition needs to be handled carefully. Since there is no next state, 
    #       you will have to pass a dummy state (like -1), which you will be using in the planning_step() to 
    #       differentiate between updates with usual terminal and non-terminal transitions.

    ### START CODE HERE ###
    
    #Part 1: last state - last action

    
    #Part 2:  Update Model  
    
    #Part 3: Update Queue
    
    # Part 4: Update predecessors

    #Part 5: Planning_step
    ### END CODE HERE ###

In [ ]:
# Do not modify this cell!

def run_experiment(env, agent, env_parameters, agent_parameters, exp_parameters):

    # Experiment settings
    num_runs = exp_parameters['num_runs']
    num_episodes = exp_parameters['num_episodes']
    planning_steps_all = agent_parameters['planning_steps']

    env_info = env_parameters                     
    agent_info = {"num_states" : agent_parameters["num_states"],  # We pass the agent the information it needs. 
                  "num_actions" : agent_parameters["num_actions"],
                  "epsilon": agent_parameters["epsilon"],
                  "theta": agent_parameters["theta"],
                  "discount": env_parameters["discount"],
                  "step_size" : agent_parameters["step_size"]}

    all_averages = np.zeros((len(planning_steps_all), num_runs, num_episodes)) # for collecting metrics 
    log_data = {'planning_steps_all' : planning_steps_all}                     # that shall be plotted later

    for idx, planning_steps in enumerate(planning_steps_all):

        print('Planning steps : ', planning_steps)
        os.system('sleep 0.5')                    # to prevent tqdm printing out-of-order before the above print()
        agent_info["planning_steps"] = planning_steps  

        for i in tqdm(range(num_runs)):

            agent_info['random_seed'] = i
            agent_info['planning_random_seed'] = i

            rl_glue = RLGlue(env, agent)          # Creates a new RLGlue experiment with the env and agent we chose above
            rl_glue.rl_init(agent_info, env_info) # We pass RLGlue what it needs to initialize the agent and environment

            for j in range(num_episodes):

                rl_glue.rl_start()                # We start an episode. Here we aren't using rl_glue.rl_episode()
                                                  # like the other assessments because we'll be requiring some 
                is_terminal = False               # data from within the episodes in some of the experiments here 
                num_steps = 0
                while not is_terminal:
                    reward, _, action, is_terminal = rl_glue.rl_step()  # The environment and agent take a step 
                    num_steps += 1                                      # and return the reward and action taken.

                all_averages[idx][i][j] = num_steps

    log_data['all_averages'] = all_averages
    np.save("results/Priority-Sweeping_steps", log_data)
    

def plot_steps_per_episode(file_path):

    data = np.load(file_path).item()
    all_averages = data['all_averages']
    planning_steps_all = data['planning_steps_all']

    for i, planning_steps in enumerate(planning_steps_all):
        plt.plot(np.mean(all_averages[i], axis=0), label='Planning steps = '+str(planning_steps))

    plt.legend(loc='upper right')
    plt.xlabel('Episodes')
    plt.ylabel('Steps\nper\nepisode', rotation=0, labelpad=40)
    plt.axhline(y=16, linestyle='--', color='grey', alpha=0.4)
    plt.show()

In [ ]:
# Do NOT modify the parameter settings.

# Experiment parameters
experiment_parameters = {
    "num_runs" : 20,                     # The number of times we run the experiment
    "num_episodes" : 30,                 # The number of episodes per experiment
}

# Environment parameters
environment_parameters = { 
    "discount": 0.95,
}

# Agent parameters
agent_parameters = {  
    "num_states" : 54,
    "num_actions" : 4, 
    "epsilon": 0.1, 
    "step_size" : 0.125,
    "theta" : 0.2,
    "planning_steps" : [25]      # The list of planning_steps we want to try
}

current_env = ShortcutMazeEnvironment   # The environment
current_agent = PriorityAgent              # The agent

run_experiment(current_env, current_agent, environment_parameters, agent_parameters, experiment_parameters)
plot_steps_per_episode('results/Priority-Sweeping_steps.npy')   
shutil.make_archive('results', 'zip', 'results');

Alright, let's begin coding.

You already know to to code Dyna-Q, you can easily execute cells of this section and jump into section 2, Dyna-Q with "Prioritized Sweeping"